# Assignment 8

In this assignment we'll estimate a regression model using the horseshoe prior. We're going to analyze [some data I found on Kaggle](https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction) and try to predict the price of flights. 

## Instructions

Please complete this Jupyter notebook and **don't** convert it to a `.py` file. Upload this notebook, along with any `.stan` files and any data sets as a `zip` file to Gradescope. Your work will be manually graded by our TA. 


### Data Cleaning

I will make some transformations to prepare this data set for regression:

 - create dummy columns for the categorical data,
 - transform price to log price,
 - create interaction terms.

Note that my decision to disregard certain columns, or interactions or transformations is a reflection of my prior knowledge. Ommitting a column is equivalent to setting a very informative prior for that hypothetical column's weight (centered at $0$). Commonly performed procedures have a place in the Bayesian philosophy, and are not as ad hoc as they seem to be in the frequentist domain!

In [ ]:
import pandas as pd
import numpy as np
import os
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt

In [ ]:
flights = pd.read_csv("Clean_Dataset.csv", index_col=0)
flights.head()

In [ ]:
# change dependent variabe by taking the logarithm
flights['log_price'] = np.log(flights['price'])
flights = flights.drop('price', axis=1)

In [ ]:
# create airlines dummies
# baseline is "other" (AirAsia","GO_FIRST", or "SpiceJet")
airline_dummies = pd.get_dummies(flights['airline']).astype(float)
airline_dummies = airline_dummies.drop(["AirAsia","GO_FIRST","SpiceJet"],axis=1)

In [ ]:
# one dummy for business class (baseline is economy)
class_dummies = pd.get_dummies(flights['class']).astype(float)
class_dummies = class_dummies['Business']
class_dummies.head()

In [ ]:
# create predictor matrix
X = pd.concat([flights[['duration', 'days_left']], airline_dummies, class_dummies], axis=1)
X.head()

## Problem 1: Running MCMC

There's a lot of data here, which means running MCMC the way we typically do is going to take a long time. Let's start by only using some of the rows. Feel free to adjust the variable below to keep as many rows as you want--the lower the number the faster your code. 

Note that your inferences will be compromised because we are throwing away data, in a sense, but we need to keep this data manageable, especially while we're getting our code ready for prime time. 

Do not run a variational algorithm yet! We'll do that in the next problem!

Note that if you keep too few rows, there's a chance that a dummy column will have all $1$s or all $0$s. This is a problem because it will introduce collinearity. Also, it will make those predictors look unimportant even when they are!

We shuffle the rows before selecting some of them to reduce the chances this happens. We also check to make sure indicator columns are not all $1$ or all $0$. If they were, the column average would be $1$ or $0$ exactly 

In [ ]:
num_rows_to_keep = 100 # change me if you want!
row_indexes = np.arange(0,X.shape[0])
np.random.shuffle(row_indexes)
rows_to_keep = row_indexes[0:num_rows_to_keep]
baby_X = X.iloc[rows_to_keep,]
baby_y = flights.loc[rows_to_keep,'log_price']
baby_X.mean(axis=0)

1.

Write `.stan` file and estimate your model using the NUTS algorithm.

In [ ]:
# code goes here

2.

Write code to check that **all** of your $\hat{R}$ convergence diagnostics. Use a threshold of $1.01$. Be honest!

You might see some warnings about "divergent transitions." This means that the geometry of the posterior is relatively complicated, and the NUTS algorithm isn't having as easy of a time with it. This is common with the horseshoe prior.

The warnings can be addressed by 

 - re-tuning the NUTS algorithm (not discussed),
 - writing the horseshoe prior differently in our `.stan` file (not discussed), or
 - **just running the NUTS sampler for longer.** 

Indeed, $\hat{R}$ will still be our metric to tell us if we didn't run our chain for long enough. If all of them are below $1.01$, then you can ignore the warnings. 



In [1]:
# code goes here

3.

What is the estimate of the intercept? What is its interpretation? What is a credible interval for this quantity? Provide an accompanying histogram.

In [ ]:
# code goes here

4.

What is the estimate of the standard deviation of the noise? What is its interpretation? What is a credible interval for this quantity? Provide an accompanying histogram.

In [ ]:
# code goes here

5.

Use a scatterplot matrix and some credible intervals to visualize the $\beta$ weight samples. Do any of the predictors look unimportant? Do any of them look correlated?



In [ ]:
# code goes here

6.

Pick an arbitrary observation number (call it $i$). This corresponds to the index of a row in your predictor matrix. 

Plot the posterior predictions for $\tilde{y}_i$ for this particular predictor. There are many simulations for this one quantity.

Compare them to the actual dependent observation at that row $y_i$. Ideally your $y_i$ should be in the center of the histogram of predictions. Overlay a vertical line to show where this $y_i$ is.

Once you're convinced you have working code, copy/paste it into a function definition so that you can quickly switch between different predictions. 

NB: `stan` indexes start at $1$, while Python indexes start at $0$!

In [ ]:
def plot_posterior_predictive_one_predictor(arb_row):
    ...
    # copy/paste code in here
    ...
#plot_posterior_predictive_one_predictor(40)

## Problem 2: Running Variational Inference Algorithms

There's a lot of data here, which means we had to ignore it to quickly run MCMC, especially while we were prototyping. 

Use a variational inference algorithm on the full data set now. Do not recompile your stan model. You don't need to. This took a few minutes on my machine.


In [ ]:
# code goes here

1.

Plot a histogram for the alpha parameter. Does it look the same as the MCMC output? 

In [ ]:
# code goes here

2.

Plot the same scatterplot matrix for the betas? How does it look compared to the full MCMC output?

In [ ]:
# code goes here

## Problem 3: MAP Estimates

It still might be a bit slow for you, even if you are just running a variational algorithm. If you would like to do as the frequentists do--just optimize a function instead of trying to find the function itself--you can just find the MAP estimate:

$$
\text{argmax}_\theta \pi(\theta \mid y)
$$

At least this way we get the benefit of the prior. This is the fastest out of the three approaches so far because there is no attempt to find the posterior's function--just its mode.

1.

Use your model's `.optimize()` function to display the point estimates for your parameters.

In [ ]:
# code goes here